
# Neural Architecture Search Example Using a tandem EfficientNetB7-ResidualMLP Model On CIFAR10


In [1]:
! pip3 install matplotlib
! pip3 install tensorflow-addons
! pip3 install pandas
! pip3 install pendulum
! pip3 install keras_tuner
import os
import pendulum
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import keras_tuner as kt
from residualmlp.residual_mlp import ResidualMLP

You should consider upgrading via the '/usr/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/usr/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/usr/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/usr/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/usr/bin/python -m pip install --upgrade pip' command.


In [2]:
# Set up distribution strategy

tf.keras.backend.clear_session()
strategy = tf.distribute.MirroredStrategy(
                                          devices=None,
                                          cross_device_ops=None)


2022-01-27 07:59:51.014352: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:1050] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-27 07:59:51.026231: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:1050] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-27 07:59:51.027081: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:1050] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-27 07:59:51.029237: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:1050] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-27 07:59:51.030167: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:1050] successful NUMA node read f

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


### Import the CIFAR 10 dataset, prepare the data for analysis, subset 5000 images for NAS use

In [3]:

TRAINING_SET_SIZE = 5000

cifar = tf.keras.datasets.cifar10.load_data()
(x_train, y_train), (x_test, y_test) = cifar

y_train_ohe = tf.one_hot([i[0] for i in  y_train],10)
indexes_for_rows = tf.range(0,y_train.shape[0])
shuffled_indexes = tf.random.shuffle(indexes_for_rows)
selected_indexes = shuffled_indexes[:TRAINING_SET_SIZE]
selected_x_train = x_train[selected_indexes,:,:,:]
selected_y_train_ohe = y_train_ohe.numpy()[selected_indexes,:]


### Load the base model:

In [4]:
BASE_MODEL_INPUT_SHAPE = (600,600,3)
with strategy.scope():
    # Base moel for transfer learning: (needs a few modifications...)
    mod_with_fc_raw = tf.keras.applications.efficientnet.EfficientNetB7(
        include_top=True, weights='imagenet', input_tensor=None,
        input_shape = BASE_MODEL_INPUT_SHAPE, pooling='max', classes=1000
    )

    # Make the deepest conv2d layer trainable, leave everything else
    # as not trainable
    for layer in mod_with_fc_raw.layers:
        layer.trainable = False
    # Last conv2d layer. This we want to train .
    mod_with_fc_raw.layers[-6].trainable = True

    # Create the final base model (remove the final Dense and BatchNormalization layers ...) 
    efficient_net_b_7_transferable_base_model =\
        tf.keras.Model(inputs=mod_with_fc_raw.layers[0].input, 
                        outputs=mod_with_fc_raw.layers[-3].output)

INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Redu

In [5]:
with strategy.scope():
    model_builder = ResidualMLP(
                        problem_type= "classification",  
                        minimum_learning_rate = 0.00007, 
                        maximum_learning_rate = .7, 
                        number_of_learning_rates_to_try = 7, 
                        input_shape = (32,32,3), 
                        bw_images = False, 
                        base_model = efficient_net_b_7_transferable_base_model, 
                        base_model_input_shape = (600,600,3), 
                        flatten_after_base_model = False, 
                        minimum_number_of_blocks = 1, 
                        maximum_number_of_blocks = 4, 
                        minimum_number_of_layers_per_block = 1, 
                        maximum_number_of_layers_per_block = 7, 
                        minimum_neurons_per_block_layer = 1, 
                        maximum_neurons_per_block_layer = 30, 
                        n_options_of_neurons_per_layer_to_try = 7, 
                        minimum_neurons_per_block_layer_decay = 0, 
                        maximum_neurons_per_block_layer_decay = 29, 
                        minimum_dropout_rate_for_bypass_layers = 0.01, 
                        maximim_dropout_rate_for_bypass_layers = .7, 
                        n_options_dropout_rate_for_bypass_layers = 7, 
                        minimum_inter_block_layers_per_block = 0, 
                        maximum_inter_block_layers_per_block = 7,  
                        n_options_inter_block_layers_per_block = 7, 
                        minimum_dropout_rate = 0.01, 
                        maximum_dropout_rate = .7,
                        n_options_dropout_rate = 7, 
                        minimum_final_dense_layers = 0,
                        maximum_final_dense_layers = 7, 
                        n_options_final_dense_layers = 7, 
                        number_of_classes = 10,
                        final_activation = tf.keras.activations.softmax)

In [6]:


date = pendulum.now().__str__()[:16].replace("T","_").replace(":","_")

RESULTS_DIR = f'CIFAR10_EfficientNetB7-ResidualMLP_{date}'
PATIENCE = 10
PATIENCE_MIN_DELTA = 0.00001
BATCH_SIZE = 300
EPOCHS = 100

logdir = os.path.join("logs", RESULTS_DIR + "_TB")
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)



2022-01-27 08:00:00.100080: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2022-01-27 08:00:00.100123: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.
2022-01-27 08:00:00.100168: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1614] Profiler found 1 GPUs
2022-01-27 08:00:00.225622: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.
2022-01-27 08:00:00.225818: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1749] CUPTI activity buffer flushed


In [7]:
# Instantiate a hyperband tuner on our model:
tuner = kt.Hyperband(
    model_builder.build_auto_residual_mlp,
    distribution_strategy = strategy,
    objective='val_loss',
    max_epochs = 30,
    hyperband_iterations = 2)


INFO:tensorflow:Reloading Oracle from existing project ./untitled_project/oracle.json
All permutations:
      number_of_blocks  layers_per_block  neurons_per_block_layer  \
0                    1                 1                        1   
1                    1                 1                        1   
2                    1                 1                        1   
3                    1                 1                        1   
4                    1                 1                        1   
...                ...               ...                      ...   
5875                 4                 7                       30   
5876                 4                 7                       30   
5877                 4                 7                       30   
5878                 4                 7                       30   
5879                 4                 7                       30   

      neurons_per_block_layer_decay  
0                            

In [8]:
tuner.search(x=selected_x_train,  
             y=selected_y_train_ohe,
             epochs=EPOCHS,
             batch_size=BATCH_SIZE, 
             callbacks=[
                    tf.keras.callbacks.EarlyStopping(
                        monitor="val_loss",
                        patience=PATIENCE,
                        min_delta=PATIENCE_MIN_DELTA,
                        restore_best_weights=True,
                    ),
                    tensorboard_callback,
                ],
             validation_split=0.3)


Search: Running Trial #2

Hyperparameter    |Value             |Best Value So Far 
learning_rate     |0.10087           |?                 
blocks            |1980              |?                 
bypass_layers_u...|0                 |?                 
inter_block_layers|7                 |?                 
final_dense_layers|2                 |?                 
b_norm_or_dropo...|bnorm             |?                 
dropout_rate_fo...|0.7               |?                 
b_norm_or_dropo...|bnorm             |?                 
dropout_rate      |0.585             |?                 
tuner/epochs      |2                 |?                 
tuner/initial_e...|0                 |?                 
tuner/bracket     |3                 |?                 
tuner/round       |0                 |?                 

All permutations:
      number_of_blocks  layers_per_block  neurons_per_block_layer  \
0                    1                 1                        1   
1                 

Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/pandas/core/indexes/range.py", line 385, in get_loc
    return self._range.index(new_key)
ValueError: 1980 is not in range

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/keras_tuner/engine/tuner.py", line 170, in _try_build
    model = self._build_hypermodel(hp)
  File "/usr/local/lib/python3.8/dist-packages/keras_tuner/engine/tuner.py", line 158, in _build_hypermodel
    model = self.hypermodel.build(hp)
  File "/notebooks/2022-01-27_NAS_Tests/residual_MLP_Tests/use-examples/CIFAR10-Neural-Architecture-Search-Example/residualmlp/residual_mlp.py", line 387, in build_auto_residual_mlp
    self.blocks = valid_permutations_df.loc[blocks_index_chosen]['blocks']
  File "/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py", line 967, in __getitem__
    return self._getitem_axis(maybe_callable, ax

All permutations:
      number_of_blocks  layers_per_block  neurons_per_block_layer  \
0                    1                 1                        1   
1                    1                 1                        1   
2                    1                 1                        1   
3                    1                 1                        1   
4                    1                 1                        1   
...                ...               ...                      ...   
5875                 4                 7                       30   
5876                 4                 7                       30   
5877                 4                 7                       30   
5878                 4                 7                       30   
5879                 4                 7                       30   

      neurons_per_block_layer_decay  
0                                 0  
1                                 1  
2                                 2  
3

Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/pandas/core/indexes/range.py", line 385, in get_loc
    return self._range.index(new_key)
ValueError: 1980 is not in range

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/keras_tuner/engine/tuner.py", line 170, in _try_build
    model = self._build_hypermodel(hp)
  File "/usr/local/lib/python3.8/dist-packages/keras_tuner/engine/tuner.py", line 158, in _build_hypermodel
    model = self.hypermodel.build(hp)
  File "/notebooks/2022-01-27_NAS_Tests/residual_MLP_Tests/use-examples/CIFAR10-Neural-Architecture-Search-Example/residualmlp/residual_mlp.py", line 387, in build_auto_residual_mlp
    self.blocks = valid_permutations_df.loc[blocks_index_chosen]['blocks']
  File "/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py", line 967, in __getitem__
    return self._getitem_axis(maybe_callable, ax

All permutations:
      number_of_blocks  layers_per_block  neurons_per_block_layer  \
0                    1                 1                        1   
1                    1                 1                        1   
2                    1                 1                        1   
3                    1                 1                        1   
4                    1                 1                        1   
...                ...               ...                      ...   
5875                 4                 7                       30   
5876                 4                 7                       30   
5877                 4                 7                       30   
5878                 4                 7                       30   
5879                 4                 7                       30   

      neurons_per_block_layer_decay  
0                                 0  
1                                 1  
2                                 2  
3

...

All permutations:
      number_of_blocks  layers_per_block  neurons_per_block_layer  \
0                    1                 1                        1   
1                    1                 1                        1   
2                    1                 1                        1   
3                    1                 1                        1   
4                    1                 1                        1   
...                ...               ...                      ...   
5875                 4                 7                       30   
5876                 4                 7                       30   
5877                 4                 7                       30   
5878                 4                 7                       30   
5879                 4                 7                       30   

      neurons_per_block_layer_decay  
0                                 0  
1                                 1  
2                                 2  
3

Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/pandas/core/indexes/range.py", line 385, in get_loc
    return self._range.index(new_key)
ValueError: 1980 is not in range

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/keras_tuner/engine/tuner.py", line 170, in _try_build
    model = self._build_hypermodel(hp)
  File "/usr/local/lib/python3.8/dist-packages/keras_tuner/engine/tuner.py", line 158, in _build_hypermodel
    model = self.hypermodel.build(hp)
  File "/notebooks/2022-01-27_NAS_Tests/residual_MLP_Tests/use-examples/CIFAR10-Neural-Architecture-Search-Example/residualmlp/residual_mlp.py", line 387, in build_auto_residual_mlp
    self.blocks = valid_permutations_df.loc[blocks_index_chosen]['blocks']
  File "/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py", line 967, in __getitem__
    return self._getitem_axis(maybe_callable, ax

All permutations:
      number_of_blocks  layers_per_block  neurons_per_block_layer  \
0                    1                 1                        1   
1                    1                 1                        1   
2                    1                 1                        1   
3                    1                 1                        1   
4                    1                 1                        1   
...                ...               ...                      ...   
5875                 4                 7                       30   
5876                 4                 7                       30   
5877                 4                 7                       30   
5878                 4                 7                       30   
5879                 4                 7                       30   

      neurons_per_block_layer_decay  
0                                 0  
1                                 1  
2                                 2  
3

Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/pandas/core/indexes/range.py", line 385, in get_loc
    return self._range.index(new_key)
ValueError: 1980 is not in range

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/keras_tuner/engine/tuner.py", line 170, in _try_build
    model = self._build_hypermodel(hp)
  File "/usr/local/lib/python3.8/dist-packages/keras_tuner/engine/tuner.py", line 158, in _build_hypermodel
    model = self.hypermodel.build(hp)
  File "/notebooks/2022-01-27_NAS_Tests/residual_MLP_Tests/use-examples/CIFAR10-Neural-Architecture-Search-Example/residualmlp/residual_mlp.py", line 387, in build_auto_residual_mlp
    self.blocks = valid_permutations_df.loc[blocks_index_chosen]['blocks']
  File "/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py", line 967, in __getitem__
    return self._getitem_axis(maybe_callable, ax

RuntimeError: Too many failed attempts to build model.